<a href="https://colab.research.google.com/github/andjelatodorovic/hypergraph_experiments/blob/main/hypergraphSBM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#TODO: optimize imports
import gzip
import json
import math
import os
from itertools import chain
import networkx as nx
import matplotlib.pyplot as plt
import pickle
from sklearn.model_selection import ShuffleSplit
import numpy as np
from scipy.sparse import csr_matrix
import networkx as nx
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
import numpy as np
import math
from itertools import chain
import os
import gzip
import json
import matplotlib.cm as cm

In [ ]:
#!pip install networkx

In [ ]:
class HSBModel:
    def __init__(
        self,
        node_community_sizes,
        edge_community_sizes,
        affinity_matrix,
        seed,
        with_hash=False,
        identifier=None,
    ):
        self.seed = seed
        self.random_state = np.random.RandomState(seed)
        self.node_community_sizes = node_community_sizes
        self.edge_community_sizes = edge_community_sizes
        self.affinity_matrix = affinity_matrix
        self.n_node_communities = len(self.node_community_sizes)
        self.n_edge_communities = len(self.edge_community_sizes)
        self.node_communities = list(
            chain.from_iterable(
                [n] * d for n, d in enumerate(self.node_community_sizes)
            ),
        )
        self.edge_communities = list(
            chain.from_iterable(
                [n] * d for n, d in enumerate(self.edge_community_sizes)
            ),
        )
        self.M_csr = self._generate_edges()
        self.M_csc = self.M_csr.tocsc()
        self.n = int(self.M_csr.shape[0])
        self.m = int(self.M_csr.shape[-1])
        self.c = int(self.M_csr.nnz)
        self.with_hash = with_hash
        self.node_degree = [int(self.M_csr[i, :].nnz) for i in range(self.n)]
        self.edge_cardinality = [int(self.M_csc[:, j].nnz) for j in range(self.m)]
        self.node_neighborhood_size = [
            len(set(self.M_csc[:, self.M_csr[i, :].nonzero()[-1]].nonzero()[0]))
            for i in range(self.M_csr.shape[0])
        ]
        self.edge_neighborhood_size = [
            len(set(self.M_csr[self.M_csc[:, j].nonzero()[0]].nonzero()[-1]))
            for j in range(self.M_csc.shape[-1])
        ]
        self.node_labels = self._generate_node_labels()
        self.identifier = identifier

    def _generate_edges(self):
        row_ind = list()
        col_ind = list()
        for node_idx, v in enumerate(self.node_communities):
            affinities = self.affinity_matrix[v, :]
            for comm_idx, (community_size, affinity) in enumerate(
                zip(self.edge_community_sizes, affinities)
            ):
                n_edges_to_sample = int(
                    self.random_state.choice([math.ceil, math.floor])(
                        affinity * community_size
                    )
                )
                if n_edges_to_sample > 0:
                    edges_to_choose_from = [
                        idx
                        for idx, e in enumerate(self.edge_communities)
                        if e == comm_idx
                    ]
                    edges_sampled = self.random_state.choice(
                        edges_to_choose_from, size=n_edges_to_sample, replace=True
                    )
                    row_ind.extend([node_idx] * len(edges_sampled))
                    col_ind.extend(edges_sampled)
        data = [1] * len(col_ind)
        # csr_matrix ignores duplicate entries
        return csr_matrix((data, (row_ind, col_ind)))

    def _generate_feature_matrix(self):
        feature_matrix = np.zeros((self.n, 1))  # adjust?
        community_params = {}
        for community in range(self.n_node_communities):
            mean = self.random_state.uniform(2, 20)
            std_dev = self.random_state.uniform(1, 3)
            community_params[community] = (mean, std_dev)
        for node_idx in range(self.n):
            community = self.node_communities[node_idx]
            mean, std_dev = community_params[community]
            #feature_matrix[node_idx, 0] = self.random_state.normal(mean, std_dev)
            feature_matrix[node_idx] = self.random_state.normal(mean, std_dev)
        return feature_matrix

    def print_hyperedges(self):
      for i in range(self.M_csr.shape[0]):
        hyperedge_nodes = self.M_csr[i].indices
        print(f"Hyperedge {i}: {hyperedge_nodes}")

    def _generate_ihg_tsv_string(self):
        c2r = [
            list(map(lambda x: int(x + 1), self.M_csc[:, c].nonzero()[0]))
            for c in range(self.M_csc.shape[-1])
        ]
        if self.identifier is None:
            return "\n".join(["\t".join(map(str, e)) for e in c2r])
        else:
            return "\n".join(
                [f"{self.identifier}\t" + "\t".join(map(str, e)) for e in c2r]
            )

    def write_ihg_tsv_gz(self, savepath):
        c2r_string = self._generate_ihg_tsv_string()
        os.makedirs(savepath, exist_ok=True)
        with gzip.open(
            f"{savepath}/{self._get_filename()}.ihg.tsv.gz", "wt", encoding="UTF-8"
        ) as zipfile:
            zipfile.write(c2r_string)

    def _generate_ihg_features(self):
        features = {
            "node_degree": self.node_degree,
            "edge_cardinality": self.edge_cardinality,
            "node_neighborhood_size": self.node_neighborhood_size,
            "edge_neighborhood_size": self.edge_neighborhood_size,
            "node_labels": self.node_labels,
            "node_communities": list(map(lambda x: x + 1, self.node_communities)),
            "edge_communities": list(map(lambda x: x + 1, self.edge_communities)),
            "config": dict(
                seed=self.seed,
                node_community_sizes=self.node_community_sizes,
                edge_community_sizes=self.edge_community_sizes,
                affinity_matrix=self.affinity_matrix.tolist(),
            ),
            "filename": self._get_filename(),
        }
        if self.identifier is not None:
            features["identifier"] = self.identifier
        return features

    def write_ihg_features_gz(self, savepath):
        with gzip.open(
            f"{savepath}/{self._get_filename()}.ihg.json.gz", "wt", encoding="UTF-8"
        ) as zipfile:
            json.dump(
                self._generate_ihg_features(),
                zipfile,
            )

    def _get_filename(self):
        return f"HSBM_n-{self.n}_m-{self.m}_c-{self.c}_nnc-{self.n_node_communities}_nmc-{self.n_edge_communities}_seed-{self.seed}{'' if not self.with_hash else '_h-' + str(hash(str(self.seed) + str(self.affinity_matrix.tolist())))}"

    def __repr__(self):
        return f"<Hypergraph Stochastic Block Model with {self.n} nodes and {self.m} edges, {self.n_node_communities} node communities and {self.n_edge_communities} edge communities>"

    '''
    def _generate_node_labels(self, noise_prob=0.1):
      labels = {}
      for node_idx, v in enumerate(self.node_communities):
          community_probabilities = self.affinity_matrix[v, :]

          # Introduce noise in label selection
          if self.random_state.rand() < noise_prob:
              # With probability `noise_prob`, choose a random label
              label = self.random_state.choice(range(len(community_probabilities)))
          else:
              # Otherwise, choose label based on community probabilities
              label = self.random_state.choice(range(len(community_probabilities)), p=community_probabilities)

          labels[node_idx] = label

      return labels
    '''

    def _generate_node_labels(self, percent_label_1 = 0.05, option = 'experiment1'):
        num_nodes = len(self.node_communities)
        num_label_1_nodes = int(percent_label_1 * num_nodes)
        node_indices = list(range(num_nodes))
        label_1_nodes = self.random_state.choice(node_indices, size=num_label_1_nodes, replace=False)

        labels = {}
        for node_idx in node_indices:
            if node_idx in label_1_nodes:
                labels[node_idx] = 1
            else:
                labels[node_idx] = 0

        # here i ensure no connected nodes have same label 1 - might be more difficult to propagate? adjust learning task?
        for node_idx in label_1_nodes:
            neighbors = np.nonzero(self.M_csr[node_idx])[1]

            for neighbor in neighbors:
                if labels[neighbor] == 1:
                    labels[node_idx] = 0
                    break

        return labels

    def _export_hyperedges(self):
        hyperedges_dict = {}
        for i in range(self.M_csr.shape[0]):
            edges = self.M_csr[i].indices
            edges = [e for e in edges if e != i]  # exclude self-loops
            hyperedges_dict[i] = edges
        return hyperedges_dict

    def generate_splits_and_export(self, n_splits=10, test_size=0.2, random_state=None, output_dir="."):
        ss = ShuffleSplit(n_splits=n_splits, test_size=test_size, random_state=random_state)
        for i, (train_index, test_index) in enumerate(ss.split(range(self.n))):
            split = {'train': train_index.tolist(), 'test': test_index.tolist()}
            output_filename = f"{output_dir}/{i+1}.pickle"
            with open(output_filename, 'wb') as file:
                pickle.dump(split, file)


    def to_bipartite_graph(self):
      G = nx.Graph()
      G.add_nodes_from(range(self.n), bipartite=0)
      G.add_nodes_from(range(self.m), bipartite=1)

      for i in range(self.M_csr.shape[0]):
          hyperedge_nodes = self.M_csr[i].indices
          for node_idx in hyperedge_nodes:
              if node_idx != i:  # Exclude self-loops
                  G.add_edge(i, node_idx)

      return G

      def print_node_labels(self):
        print("Node Labels:")
        for node_idx, label in self.node_labels.items():
            print(f"Node {node_idx}: Label {label}")



    def visualize_bipartite_graph(self):
        G = nx.Graph()
        hyperedge_nodes = set()
        original_nodes = set()

        for i in range(self.M_csr.shape[0]):
            hyperedge_id = f"{i}e"
            hyperedge_nodes.add(hyperedge_id)
            for node_idx in self.M_csr[i].indices:
                original_node_id = f"{node_idx}n"
                original_nodes.add(original_node_id)
                G.add_edge(hyperedge_id, original_node_id)

        #sort for better visibility
        hyperedge_nodes_sorted = sorted(hyperedge_nodes, key=lambda x: int(x[:-1]), reverse=True)
        original_nodes_sorted = sorted(original_nodes, key=lambda x: int(x[:-1]), reverse=True)


        pos = {}
        for i, node_id in enumerate(hyperedge_nodes_sorted):
            pos[node_id] = (0, i)
        for j, node_id in enumerate(original_nodes_sorted):
            pos[node_id] = (1, j)

        plt.figure(figsize=(8, 6))
        node_colors = ['orange' if node_id.endswith('e') else 'lightpink' for node_id in G.nodes()]
        labels = {node_id: node_id for node_id in G.nodes()}
        nx.draw(G, pos, with_labels=True, labels=labels, node_color=node_colors, node_size=300, font_size=8)
        plt.title("Bipartite Graph Visualization")
        plt.show()


    #vary based on the node community

    def visualize_bipartite_graph(self):
      G = nx.Graph()
      hyperedge_nodes = set()
      original_nodes = set()

      for i in range(self.M_csr.shape[0]):
          hyperedge_id = f"{i}e"
          hyperedge_nodes.add(hyperedge_id)
          for node_idx in self.M_csr[i].indices:
              original_node_id = f"{node_idx}n"
              original_nodes.add(original_node_id)
              G.add_edge(hyperedge_id, original_node_id)

      # Sort nodes for better visibility
      hyperedge_nodes_sorted = sorted(hyperedge_nodes, key=lambda x: int(x[:-1]), reverse=True)
      original_nodes_sorted = sorted(original_nodes, key=lambda x: int(x[:-1]), reverse=True)

      pos = {}
      for i, node_id in enumerate(hyperedge_nodes_sorted):
          pos[node_id] = (0, i)
      for j, node_id in enumerate(original_nodes_sorted):
          pos[node_id] = (1, j)

      plt.figure(figsize=(10, 8))

      # Create node color mapping based on node communities
      if len(G.original_nodes()) != len(self.node_communities):
          print("Error: Number of nodes in G does not match length of node_communities")
          return

      unique_communities = set(self.node_communities)
      color_map = cm.get_cmap('tab10', len(unique_communities))  # Choose a colormap
      community_color_dict = {c: color_map(i) for i, c in enumerate(unique_communities)}

      # Assign node colors based on node communities
      node_colors = []
      for node_id in G.nodes():
          if node_id.endswith('n'):
              node_idx = int(node_id[:-1])
              if node_idx < len(self.node_communities):
                  community_label = self.node_communities[node_idx]
                  if community_label in community_color_dict:
                      node_colors.append(community_color_dict[community_label])
                  else:
                      node_colors.append('gray')  # Default color for unknown community
              else:
                  node_colors.append('gray')  # Default color for out-of-range index
          else:
              node_colors.append('gray')  # Default color for hyperedge nodes (shouldn't happen)

      # Draw nodes and edges
      nx.draw(G, pos, with_labels=True, node_color=node_colors, node_size=300, font_size=8, cmap=color_map)

      plt.title("Bipartite Graph Visualization")
    plt.show()

In [ ]:
node_community_sizes = [4, 4, 4]
edge_community_sizes = [5, 5, 5]
affinity_matrix = np.random.rand(len(node_community_sizes), len(edge_community_sizes))
affinity_matrix = affinity_matrix / affinity_matrix.sum(axis=1)[:, np.newaxis]

affinity_matrix = np.eye(3,3) * 0.2 + np.random.uniform(0,0.3) #adding noise but ensuring the sparsity is perserved
seed = 42

#change affinity matrix to be diagonally dominant

model = HSBModel(node_community_sizes, edge_community_sizes, affinity_matrix, seed)
print(model.affinity_matrix)
model.print_hyperedges()
model.visualize_bipartite_graph()

[[0.45950428 0.25950428 0.25950428]
 [0.25950428 0.45950428 0.25950428]
 [0.25950428 0.25950428 0.45950428]]
Hyperedge 0: [ 2  3  4  9 11 12]
Hyperedge 1: [ 2  4  7 14]
Hyperedge 2: [ 1  3  8 10 13]
Hyperedge 3: [ 3  4  5  7 11 13]
Hyperedge 4: [ 2  5  8 12 14]
Hyperedge 5: [ 0  4  5  6  8 11]
Hyperedge 6: [ 0  6  9 13]
Hyperedge 7: [ 3  5  7  8 13]
Hyperedge 8: [ 1  9 11 13]
Hyperedge 9: [ 1  8 10 13]
Hyperedge 10: [ 4  9 11 14]
Hyperedge 11: [ 3  9 10 14]
Error: Number of nodes in G does not match length of node_communities


<Figure size 1000x800 with 0 Axes>

In [ ]:
def compute_edge_cardinalities(M_csc):
    edge_cardinalities = [int(M_csc[:, j].nnz) for j in range(M_csc.shape[-1])]
    return edge_cardinalities

def compute_edge_neighborhood_sizes(M_csr, M_csc):
    edge_neighborhood_sizes = [
            len(set(M_csr[M_csc[:, j].nonzero()[0]].nonzero()[-1]))
            for j in range(M_csc.shape[-1])
        ]
    return edge_neighborhood_sizes

def compute_node_degrees(M_csr):
    node_degrees = [int(M_csr[i, :].nnz) for i in range(M_csr.shape[0])]
    return node_degrees

def compute_node_neighborhood_sizes(M_csr, M_csc):
    node_neighborhood_sizes = [
            len(set(M_csc[:, M_csr[i, :].nonzero()[-1]].nonzero()[0]))
            for i in range(M_csr.shape[0])
        ]
    return node_neighborhood_sizes

In [ ]:
import numpy as np
import pickle

def main():
    # Example parameters
    # num_nodes = 10000
    # num_communities = 5

    # # Generate node_community_sizes and edge_community_sizes
    # node_community_sizes = np.random.randint(50, 200, size=num_communities)
    # edge_community_sizes = np.random.randint(10, 50, size=num_communities)
    #Generate a random affinity matrix
    # affinity_matrix = np.random.rand(num_communities, num_communities)

    node_community_sizes = [50, 50, 50]
    edge_community_sizes = [10, 10, 10]
    affinity_matrix = np.random.rand(len(node_community_sizes), len(edge_community_sizes))
    affinity_matrix = affinity_matrix / affinity_matrix.sum(axis=1)[:, np.newaxis]
    seed = 42

    model = HSBModel(node_community_sizes, edge_community_sizes, affinity_matrix, seed)


    #can this be generalized to different values for edge and node comm sizes?

    affinity_matrix = affinity_matrix / affinity_matrix.sum(axis=1)[:, np.newaxis]
    print(affinity_matrix)

    #check the symmetry here for the affinity matrix? and the meaning of the affinity

    seed = 42

    # Create HSBModel instance
    model = HSBModel(
        node_community_sizes=node_community_sizes,
        edge_community_sizes=edge_community_sizes,
        affinity_matrix=affinity_matrix,
        seed=seed,
        with_hash=False,
        identifier=None
    )

    # Generate and export splits
    model.generate_splits_and_export(n_splits=10, test_size=0.2, output_dir="splits")

    # Print IHG features
    ihg_features = model._generate_ihg_features()
    edges = model.M_csr.nonzero()
    labels = model._generate_node_labels()
    features = model._generate_feature_matrix()
    hyperedges_dict = model._export_hyperedges()

    # Save hyperedges, labels, and features to pickle files
    with open("hypergraph.pickle", 'wb') as f_hyperedges:
        pickle.dump(model._export_hyperedges(), f_hyperedges)

    with open(f"labels.pickle", 'wb') as f_labels:
        pickle.dump(list(model.node_labels.values()), f_labels)

    with open(f"features.pickle", 'wb') as f_matrix:
        pickle.dump(model._generate_feature_matrix(), f_matrix)

    print(model.__repr__())
    print("\nHSBModel instance created and files printed successfully.")

if __name__ == "__main__":
    main()


[[0.03687163 0.75014885 0.21297952]
 [0.24095868 0.35649626 0.40254506]
 [0.11960581 0.15341057 0.72698362]]
<Hypergraph Stochastic Block Model with 150 nodes and 30 edges, 3 node communities and 3 edge communities>

HSBModel instance created and files printed successfully.


In [ ]:
import pickle
from sklearn.model_selection import ShuffleSplit

def generate_splits_and_export(n, n_splits=10, test_size=0.2, random_state=None, output_dir="."):
    ss = ShuffleSplit(n_splits=n_splits, test_size=test_size, random_state=random_state)
    for i, (train_index, test_index) in enumerate(ss.split(range(n))):
        split = {'train': train_index.tolist(), 'test': test_index.tolist()}
        output_filename = f"{output_dir}/split_{i}.pkl"
        with open(output_filename, 'wb') as file:
            pickle.dump(split, file)
        print(f"Split {i} exported to {output_filename}")

In [ ]:
import torch, math, numpy as np, scipy.sparse as sp
import torch.nn as nn, torch.nn.functional as F, torch.nn.init as init

from torch.autograd import Variable
from torch.nn.modules.module import Module
from torch.nn.parameter import Parameter


class HyperGCN(nn.Module):
    def __init__(self, nfeat, nhid, nclass, nlayer, V, E, X):
        """
        d: initial node-feature dimension
        h: number of hidden units
        c: number of classes
        """
        super(HyperGCN, self).__init__()
        d, l, c = nfeat, nlayer, nclass
        cuda = False
        mediators = True
        fast = False

        h = [d]
        for i in range(l-1):
            power = l - i + 2
            h.append(2**power)
        h.append(c)

        if fast:
            reapproximate = False
            structure = Laplacian(V, E, X, mediators)
        else:
            reapproximate = True
            structure = E

        self.layers = nn.ModuleList([HyperGraphConvolution(h[i], h[i+1], reapproximate, cuda) for i in range(l)])
        self.do, self.l = 0.5, 3
        self.structure, self.m = structure, mediators



    def forward(self, H):
        """
        an l-layer GCN
        """
        do, l, m = self.do, self.l, self.m

        for i, hidden in enumerate(self.layers):
            H = F.relu(hidden(self.structure, H, m))
            if i < l - 1:
                V = H
                H = F.dropout(H, do, training=self.training)

        return F.log_softmax(H, dim=1)

class HyperGraphConvolution(Module):
    """
    Simple GCN layer, similar to https://arxiv.org/abs/1609.02907
    """

    def __init__(self, a, b, reapproximate=True, cuda=True):
        super(HyperGraphConvolution, self).__init__()
        self.a, self.b = a, b
        self.reapproximate, self.cuda = reapproximate, cuda

        self.W = Parameter(torch.FloatTensor(a, b))
        self.bias = Parameter(torch.FloatTensor(b))
        self.reset_parameters()



    def reset_parameters(self):
        std = 1. / math.sqrt(self.W.size(1))
        self.W.data.uniform_(-std, std)
        self.bias.data.uniform_(-std, std)



    def forward(self, structure, H, m=True):
        W, b = self.W, self.bias
        HW = torch.mm(H, W)

        if self.reapproximate:
            n, X = H.shape[0], HW.cpu().detach().numpy()
            A = Laplacian(n, structure, X, m)
        else: A = structure

        if self.cuda: A = A.cuda()
        A = Variable(A)

        AHW = SparseMM.apply(A, HW)
        return AHW + b



    def __repr__(self):
        return self.__class__.__name__ + ' (' \
               + str(self.a) + ' -> ' \
               + str(self.b) + ')'



class SparseMM(torch.autograd.Function):
    """
    Sparse x dense matrix multiplication with autograd support.
    Implementation by Soumith Chintala:
    https://discuss.pytorch.org/t/
    does-pytorch-support-autograd-on-sparse-matrix/6156/7
    """
    @staticmethod
    def forward(ctx, M1, M2):
        ctx.save_for_backward(M1, M2)
        return torch.mm(M1, M2)

    @staticmethod
    def backward(ctx, g):
        M1, M2 = ctx.saved_tensors
        g1 = g2 = None

        if ctx.needs_input_grad[0]:
            g1 = torch.mm(g, M2.t())

        if ctx.needs_input_grad[1]:
            g2 = torch.mm(M1.t(), g)

        return g1, g2



def Laplacian(V, E, X, m):
    """
    approximates the E defined by the E Laplacian with/without mediators

    arguments:
    V: number of vertices
    E: dictionary of hyperedges (key: hyperedge, value: list/set of hypernodes)
    X: features on the vertices
    m: True gives Laplacian with mediators, while False gives without

    A: adjacency matrix of the graph approximation
    returns:
    updated data with 'graph' as a key and its value the approximated hypergraph
    """

    edges, weights = [], {}
    rv = np.random.rand(X.shape[1])

    for k in E.keys():
        hyperedge = list(E[k])

        p = np.dot(X[hyperedge], rv)   #projection onto a random vector rv
        s, i = np.argmax(p), np.argmin(p)
        Se, Ie = hyperedge[s], hyperedge[i]

        # two stars with mediators
        c = 2*len(hyperedge) - 3    # normalisation constant
        if m:

            # connect the supremum (Se) with the infimum (Ie)
            edges.extend([[Se, Ie], [Ie, Se]])

            if (Se,Ie) not in weights:
                weights[(Se,Ie)] = 0
            weights[(Se,Ie)] += float(1/c)

            if (Ie,Se) not in weights:
                weights[(Ie,Se)] = 0
            weights[(Ie,Se)] += float(1/c)

            # connect the supremum (Se) and the infimum (Ie) with each mediator
            for mediator in hyperedge:
                if mediator != Se and mediator != Ie:
                    edges.extend([[Se,mediator], [Ie,mediator], [mediator,Se], [mediator,Ie]])
                    weights = update(Se, Ie, mediator, weights, c)
        else:
            edges.extend([[Se,Ie], [Ie,Se]])
            e = len(hyperedge)

            if (Se,Ie) not in weights:
                weights[(Se,Ie)] = 0
            weights[(Se,Ie)] += float(1/e)

            if (Ie,Se) not in weights:
                weights[(Ie,Se)] = 0
            weights[(Ie,Se)] += float(1/e)

    return adjacency(edges, weights, V)



def update(Se, Ie, mediator, weights, c):
    """
    updates the weight on {Se,mediator} and {Ie,mediator}
    """

    if (Se,mediator) not in weights:
        weights[(Se,mediator)] = 0
    weights[(Se,mediator)] += float(1/c)

    if (Ie,mediator) not in weights:
        weights[(Ie,mediator)] = 0
    weights[(Ie,mediator)] += float(1/c)

    if (mediator,Se) not in weights:
        weights[(mediator,Se)] = 0
    weights[(mediator,Se)] += float(1/c)

    if (mediator,Ie) not in weights:
        weights[(mediator,Ie)] = 0
    weights[(mediator,Ie)] += float(1/c)

    return weights



def adjacency(edges, weights, n):
    """
    computes an sparse adjacency matrix

    arguments:
    edges: list of pairs
    weights: dictionary of edge weights (key: tuple representing edge, value: weight on the edge)
    n: number of nodes

    returns: a scipy.sparse adjacency matrix with unit weight self loops for edges with the given weights
    """

    dictionary = {tuple(item): index for index, item in enumerate(edges)}
    edges = [list(itm) for itm in dictionary.keys()]
    organised = []

    for e in edges:
        i,j = e[0],e[1]
        w = weights[(i,j)]
        organised.append(w)

    edges, weights = np.array(edges), np.array(organised)
    adj = sp.coo_matrix((weights, (edges[:, 0], edges[:, 1])), shape=(n, n), dtype=np.float32)
    adj = adj + sp.eye(n)

    A = symnormalise(sp.csr_matrix(adj, dtype=np.float32))
    A = ssm2tst(A)
    return A



def symnormalise(M):
    """
    symmetrically normalise sparse matrix

    arguments:
    M: scipy sparse matrix

    returns:
    D^{-1/2} M D^{-1/2}
    where D is the diagonal node-degree matrix
    """

    d = np.array(M.sum(1))

    dhi = np.power(d, -1/2).flatten()
    dhi[np.isinf(dhi)] = 0.
    DHI = sp.diags(dhi)    # D half inverse i.e. D^{-1/2}

    return (DHI.dot(M)).dot(DHI)



def ssm2tst(M):
    """
    converts a scipy sparse matrix (ssm) to a torch sparse tensor (tst)

    arguments:
    M: scipy sparse matrix

    returns:
    a torch sparse tensor of M
    """

    M = M.tocoo().astype(np.float32)

    indices = torch.from_numpy(np.vstack((M.row, M.col))).long()
    values = torch.from_numpy(M.data)
    shape = torch.Size(M.shape)

    return torch.sparse.FloatTensor(indices, values, shape)